In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
import plotly.express as px
from tqdm import tqdm
import random
import os

import optuna
from optuna.samplers import RandomSampler
from optuna.samplers import TPESampler

optuna.logging.set_verbosity(optuna.logging.ERROR)

In [9]:
# !/usr/bin/python3 -m pip install holidays

In [8]:
import sys
sys.path.append('../src')
sys.path.append('../inference_incomes')

import vrpp
import predict
import importlib
importlib.reload(vrpp)
importlib.reload(predict)

<module 'predict' from '/home/chervovn04/Programming/hackathons/2023/optimal_encashement/notebooks/../inference_incomes/predict.py'>

In [10]:
dist = pd.read_csv('../data/raw/times v4.csv')
le = preprocessing.LabelEncoder()
le.fit(dist['Origin_tid'])
dist['from_int'] = le.transform(dist['Origin_tid'])
dist['to_int'] = le.transform(dist['Destination_tid'])
dist.head()

,Origin_tid,Destination_tid,Total_Time,from_int,to_int
0,636538,683103,15.32,864,1354
1,636538,634763,16.20,864,624
2,636538,683128,16.27,864,1358
3,636538,683789,16.77,864,1374
4,636538,634709,17.67,864,603


In [11]:
data = pd.read_excel('../data/raw/terminal_data_hackathon v4.xlsx', 'Incomes')
data.head()

,TID,остаток на 31.08.2022 (входящий),2022-09-01 00:00:00,2022-09-02 00:00:00,2022-09-03 00:00:00,2022-09-04 00:00:00,2022-09-05 00:00:00,2022-09-06 00:00:00,2022-09-07 00:00:00,2022-09-08 00:00:00,...,2022-11-21 00:00:00,2022-11-22 00:00:00,2022-11-23 00:00:00,2022-11-24 00:00:00,2022-11-25 00:00:00,2022-11-26 00:00:00,2022-11-27 00:00:00,2022-11-28 00:00:00,2022-11-29 00:00:00,2022-11-30 00:00:00
0,406136,160000,90000,105000,99000,107000,110000,60000,75000,89000,...,91000,78000,0,165000,0,189000,106000,94000,75000,74000
1,406139,387000,103000,206000,168000,124000,78000,165000,164000,174000,...,164000,153000,151000,157000,206000,182000,123000,138000,112000,179000
2,406145,287000,143000,136000,124000,117000,123000,140000,139000,138000,...,119000,100000,179000,169000,118000,118000,114000,128000,121000,124000
3,406148,355000,50000,73000,53000,65000,75000,100000,53000,52000,...,48000,55000,65000,85000,95000,68000,62000,0,118000,70000
4,406180,597000,96000,82000,71000,72000,86000,55000,55000,75000,...,82000,56000,70000,59000,105000,70000,77000,87000,59000,55000


In [12]:
config = {'num_terminals': dist['from_int'].max() + 1,
          'persent_day_income': 0.02 / 365,
          'terminal_service_cost': 100,
          'terminal_service_persent': 0, #0.01
          'max_terminal_money': 1000000,
          'max_not_service_days': 14,
          'armored_car_day_cost': 20000,
          'work_time': 10 * 60,
          'service_time': 10,
          'left_days_coef': 0,
          'encashment_coef': 0,}

In [42]:
cash_predicted = predict.proccessing('../data/raw/terminal_data_hackathon v4.xlsx', 
                    model_path='../inference_incomes/catboost.pkl', 
                    tid_path='../inference_incomes/tid_mean.pkl')
cash_predicted = cash_predicted.to_numpy()[:, 1:]

/home/chervovn04/Programming/hackathons/2023/optimal_encashement/notebooks/../inference_incomes/predict.py:172: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  res[dt].iloc[i] = a[f"{res['TID'].iloc[i]}-{dt}"]


In [43]:
# real_cash = (data[data.columns[1:]].values * (1 + 0.2 * (random.random() - 0.5))).copy()
real_cash = data[data.columns[1:]].values.copy()

In [48]:
real_cash

array([[160000,  90000, 105000, ...,  94000,  75000,  74000],
       [387000, 103000, 206000, ..., 138000, 112000, 179000],
       [287000, 143000, 136000, ..., 128000, 121000, 124000],
       ...,
       [ 67000,  63000,  63000, ...,  47000,  62000,  50000],
       [278000,  63000,      0, ...,      0,  85000,  45000],
       [333000,      0,      0, ...,      0,  55000,      0]])

In [44]:
INF = 1e9

In [45]:
def get_cost(days_left):
    if days_left == 0: return INF
    return 2 ** (config['max_not_service_days'] - days_left)

In [46]:
cash_predicted

array([[160000.        , 124266.39591377, 100105.50625945, ...,
         91397.94949465,  88876.7086165 ,  89224.84291377],
       [387000.        , 156475.63530542, 146028.45589145, ...,
        153659.64108095, 146512.92992753, 165325.32982196],
       [287000.        , 157959.72126009, 146187.13986131, ...,
        121071.67586057, 142850.40502764, 136664.07839765],
       ...,
       [ 67000.        ,  72232.30348118,  62234.62744782, ...,
         54510.70328354,  55185.13796014,  54950.28360522],
       [278000.        ,  42147.16601651,  26703.8406202 , ...,
         20366.13827356,  33608.76271713,  17839.36532031],
       [333000.        ,  28376.87222289,  32855.38414288, ...,
         15013.61041688,  25454.6699013 ,  15613.90927711]])

In [47]:
cash = cash_predicted.copy()
num_vehicles = 5
num_terminals = cash.shape[0]
time_until_force = [config['max_not_service_days'] for i in range(num_terminals)]
print(cash[:, 0].shape)
cur_cash = cash[:, 0]
cash = cash[:, 1:]

hist = {'visited': [], 'loss': []}
bad = False

myvrp = vrpp.VRPP(dist, 10, 10 * 60, num_vehicles, solution_limit=100, time_limit=100, dead_loss=False)

days = 90
for day in tqdm(range(days)):
    mask = []
    cost = []
    to_counter = [0 for i in range(config['max_not_service_days'] + 1)]
    
    for i in range(num_terminals):
        force = time_until_force[i]
        if cur_cash[i] > config['max_terminal_money']:
            force = 0
        else if cur_cash[i] + cash[i][day] > config['max_terminal_money']:
            force = 1

        force_for_show = time_until_force[i]
        current_money = cur_cash[i]
        for forecast in range(min(force_for_show, days - day)):
            if current_money > config['max_terminal_money']:
                force_for_show = forecast
                break
            if not forecast:
                current_money += cash[i][day + forecast]
            else:
                current_money += cash[i][day + forecast] # check later
                
        mask.append(1)
        cost.append(int(get_cost(force)))
        to_counter[force_for_show] += 1
    
    print(to_counter)
    # # for i in range(100):
    # #     print(" " * (4 - len(str(cost[i]))) + str(cost[i]), end=' ')
    # # print()
    # print(" "cost[:100])
    
    visited, paths = myvrp.find_vrp(cost, mask)
    hist['visited'].append(visited)
    
    prev = []
    cur_loss = 0
    for i in range(num_terminals):
        if cur_cash[i] > config['max_terminal_money'] or time_until_force[i] == 0:
            if not visited[i]:
                print('Dead')
                bad = True
                break

        if visited[i]:
            cur_loss += max(config['terminal_service_cost'], cur_cash[i] * config['terminal_service_persent'])
            cur_cash[i] = 0
            time_until_force[i] = config['max_not_service_days']
        else:
            time_until_force[i] -= 1
            
        cur_loss += cur_cash[i] * config['persent_day_income']
        cur_cash[i] += cash[i][day]
    
    if bad:
        break
    
    hist['loss'].append(cur_loss)
    
print(sum(hist['loss']))

(1630,)


  0%|                                                                             | 0/90 [00:00<?, ?it/s]

[11, 7, 24, 37, 32, 46, 64, 66, 73, 76, 96, 73, 75, 74, 876]


  1%|▋                                                                  | 1/90 [00:41<1:01:46, 41.65s/it]

[0, 21, 35, 28, 42, 53, 66, 70, 70, 96, 69, 74, 79, 774, 153]


...

[0, 56, 85, 92, 101, 124, 107, 143, 134, 121, 125, 140, 133, 133, 136]


 99%|███████████████████████████████████████████████████████████████████▏| 89/90 [51:34<00:33, 33.79s/it]

[0, 11, 84, 101, 123, 106, 143, 134, 120, 125, 140, 133, 133, 136, 141]


100%|████████████████████████████████████████████████████████████████████| 90/90 [52:28<00:00, 34.98s/it]

3557926.1168750892
